### Agent-Lab: Multi-Agent -> Browser

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest


os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Anthropic Browser

In [2]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-sonnet-4-0", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_browser_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_browser_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="Using web browser at your disposal, visit https://en.wikipedia.org/wiki/Mathematical_finance and rewrite the first paragraph in a simpler way that a 12 year old would understand.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_browser_agent.get_input_params(message, schema="public")
config = anthropic_browser_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_browser_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user wants me to visit a Wikipedia page about Mathematical finance using a web browser and then rewrite the first paragraph in simpler language suitable for a 12-year-old. This involves two main tasks: 1) Using the browser agent to visit the Wikipedia page and extract the first paragraph, and 2) Using the reporter agent to rewrite that content in simpler terms for a young audience.",
  "title": "Simplify Mathematical Finance Wikipedia Article for 12-Year-Olds",
  "steps": [
    {
      "agent_name": "browser",
      "title": "Visit Wikipedia page and extract first paragraph",
      "description": "Navigate to https://en.wikipedia.org/wiki/Mathematical_finance using the web browser. Locate and extract the complete first paragraph of the article. The browser should capture the exact text content of the opening paragraph to ensure accurate information for subsequent simplification."
    },
    {
      "agent_name": "reporter",
      "title": "Rewrite paragraph for 12-year-old comprehension",
      "description": "Take the extracted first paragraph from the Mathematical Finance Wikipedia page and rewrite it in simple, clear language that a 12-year-old can easily understand. Use basic vocabulary, shorter sentences, relatable examples, and avoid complex financial or mathematical jargon. The goal is to convey the core concept of what mathematical finance is about in an age-appropriate way."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Now let me rewrite this first paragraph in a way that's much easier for a 12-year-old to understand:

**Simplified version for a 12-year-old:**

Mathematical finance is like using math to understand money and how it works. It's when smart people use numbers and math formulas to figure out money problems. There are two main ways people use this math with money: First, they figure out how much certain money deals (like special contracts) should cost. Second, they use math to help people decide how to invest their money safely and make the best choices about where to put their money so they don't lose it.

**Key changes made:**
- Replaced "field of applied mathematics" with "using math to understand money"
- Explained "mathematical modeling" as "using numbers and math formulas to figure out money problems"
- Simplified "derivatives pricing" to "figure out how much certain money deals should cost"
- Changed "risk and portfolio management" to "help people invest their money safely and make the best choices"
- Used shorter, simpler sentences
- Removed technical jargon like "quantitative finance" and "advanced quantitative techniques"